# System Design Notes

## System Design Primer

### Basic steps

#### 1. Outline use cases, constrains & assumptions

* Who is going to use it?
* How are they going to use it?
* How many users are there?
* What does the system do?
* What are the inputs & outputs?
* How much data do we expect to handle?
* How many requests per second do we expect
* How many reads per seconds do we expect?

#### 2. Create a high-level design

* Sketch the main components and connections

#### 3. Design the core components

* Dive into details for each core component.

#### 4. Scale the design

* Identify the bottlenecks given the contraints.
    * E.g., load balancing, horizontal scaling, sharding, caching
    

### Core Concepts

#### Vertical Scaling

* Add more CPU, RAM, etc to a server
* Hit technological limits fast, and expensive

#### Horizontal scaling

* Buy more, cheaper servers
* Route users to a particular server using a load balance

_Now we run into the "sticky session" problem..._

#### Load Balancing

* Decides which server to send users to
  * Round robin
  * Route based on user name / IP hash
  * Route based on server load

_Now we need to know where the user's session data is..._

#### Sticky sessions / Distributed sessions

##### Sticky Sessions

* Using IP, cookie etc, always send user to the same server
+ No need to pass information between servers
+ Generally faster, since all data in one server
- Servers can become overloaded, eg. if we had on IP and many users come from similar IP
- If the server goes down, you have to shift all of the user session data to another server

##### Distributed Sessions

* Set up a shared cache like Redis that all servers can share.
+ Can load balance each request, rather than tying users/sessions to a server
+ No problem if server goes down -- session info stored in a distributed cache
- Slower, since we have to call over the network to the cache
- More complicated architecture

_Now we have all of our session and DB data spread across multiple databases..._

#### Replication

Our load balancers / database / session database could become a single point of failure.

* Load balancers
  * Active-active replication: Send a heartbeat between the two load balancers
  * If stop receiving a heartbeat, one is down and needs to be replaced
  
* Databases
  * Master-slave
    * Master database writes to slaves
    * If master goes down, a slave can be promoted to master
    * Slaves can also be used for reads, since data is replicated to them
    
#### Latency vs. Throughput

__Latency__: Time to perform some action

__Throughput__: The number of actions per unit of time

Generally we should aim for _maximal throughput_ with _acceptable latency_.

#### CAP Theorem

CAP:

* __Consistency__: Every read receives the most recent write or error
* __Availability__: Every request receives a response, without the guarantee that is it the most recent
* __Partition Tolerance__: The system continues to operate despite arbitrary partitioning due to network failures

Since networks are by nature unreliable, we need to support partition tolerance. This leaves us with two options: consistency vs. availability.

##### CP: Consistency & Partition tolerance

Waiting for response might result in time out. But this is a good choice if you need atomic reads and writes.

##### AP: Availability & Partition tolerance

Responses return most readily available version of the data, but it might not be the latest. Writes might take some time to propogate. But this is a good choice when the system requires eventual consistency.

#### Consistency Patterns

##### Weak Consistency

After a write, reads may or may not see it. A best effort approach.

Works well for things like VoIP, video chat, games. For example, if your connection lags during a call, you don't hear what was missed during the connection lag.

##### Eventual Consistency

After a write, reads will eventually see it. Data is replicated asynchronously. Seen in DNS and email. Works well in highly available systems.

##### Strong Consistency

After a write, reads see it. Data is replicated synchronously. Seen in file systems. Works well for systems that need transactions.


#### Availability Patterns

##### Failover: 1. Active-passive

Heartbeats sent between active and passive server on standby. If the heartbeat is interrupted, the passive server takes over and resume service.

##### Failover: 2. Active-active

Both servers manage traffic.

##### Failure: Disadvantages

* Adds more hardware and additional complexity
* Potential data loss if active system fails before new data can be replicated


Q: How is DNS used in system design? Just for routing external traffic from our load balancer to our servers?

### DNS Recap


>> Local Browser  --> Requests google.com IP, not in local cache
>> ISP DNS Server --> Requests google.com IP, not in local cache
>> ROOT DNS Server --> Finds google.com IP, passes down to the lower servers

### CDN Recap

A globally distributed network of proxy servers, for serving content closer to the user.

Usually they server static content such as HTML, CSS, JS, media etc. Though some serve dynamic content.

Using a CNS means assets can be served by closer servers and your backend servers don't have to fulfil these requests.

#### Push CDNs

Receive new content whenever changes occur on your server. Your server has responsibility to push the content to the CDN server and rewrite URLs to point to the CDN.

Sites with a small amount / non-frequently changing content suit push CDNs.

#### Pull CDNs

Grab new content from your server when the first user requests the content. A TLL defines how long the content lives before it is refreshed.

This results in a slow initial request, but is good for sites with heavy traffic and frequently changing content.

#### CDN Disadvantages

* Extra cost
* Content could become stale on the CDN server
* Require static content URLs to point to CDN URLs

### Databases

#### Relational Databases

A collection of items represented by tables and associated by keys.

__ACID__ is a set of properties for relational database transactions.

* __Atomicity__: Each transaction is all or nothing
* __Consistency__: Any transaction will bring the database from one valid state to another
* __Isolation__: Executing transactions concurrently has the same result as if they were executed serially
* __Durability__: Once a transaction has been committed, it will remain so

##### Master-slave Replication

The master serves writes and reads, replicates to slaves, and the slaves serve writes.

##### Master-master Replication

Two masters, that both serve reads and writes.

Can have write latency due to synchronization.

##### Federation

Splitting DBs into functions. For example, creating a User DB, a Products DB, etc.

This means we can fit more data in the DB and write to different databases in parallel. However, it adds complexity and requires joining databases, which can be inefficient.

##### Sharding

Distributing data across different databases. For example, users A-C in DB1, users D-F in DB2, etc.

This means we can store more data across our DBs; however, it adds complexity and certain shards could become overloaded.

#### NoSQL Databases

NoSQL databases store denormalized data. Rather than ACID transactions, NoSQL DBs favor eventual consistency.

There are 4 main types of NoSQL DBs:

##### Key-value Stores

_Abstraction: Hash table_

Allows O(1) reads and writes. Maintains keys in lexicographic order for efficient retrieval of key values.

##### Document store

_Abstraction: Key-value store with documents as the values._

Stores documents as the values, where documents can be XML, JSON, binary, etc. Provide some API or query language to query based on the internal structure of the document. Examples: MongoDB, CouchDB.

##### Wide-column store

_Abstraction: nested map (ColumnFamily<RowKey, Columns<ColKey, Value>>

Basic data unit is a column (name/value pair). Columns are grouped into column families. Super column families group column families. You can access each row with a key, and then individual elements within the column in that row.
Examples: Google Bigtable, Cassandra.

![title](img/sd4.jpg)

##### Graph DB

Each node is a record and arcs represent relationships between two nodes. Not widely used.

#### SQL vs NoSQL

##### SQL

* Structured data
* Strict schema
* Transactions
* Indexing is fast
* But need for complex joins

E.g. Cloud Spanner, BigQuery

##### NoSQL

* Semi-structured
* Flexible schema, non-relational
* No need for joins, so easier to horizontally scale -- everything is JSON object
* Eventual consistency (BASE): Data is replicated so will eventually be up-to-date. Maybe doesn't matter for things like Facebook posts, but might matter for things like email or orders

E.g. Bigtable

__Example__

```
Users
- alovelace
  - first: Ada
  - last: Lovelace
- sride
  - first: Sally
  - last: Ride
  
Rooms
- software
  - messages
    - message_1
      - from: alovelace
      - to: sride
      - message: ...
```

### Caching

#### Types of cache

* __Client caching__: Cache on OS or browser
* __CDN caching__
* __Database caching__
* __Application caching__: For example, memcache or redis

### Message queues

If the job is too slow to process inline, you can use a message queue as follows:

* An application puts the job in the queue and notifies the user of the stats
* A worker picks up the job, processes it, and signals when it is complete

The user is not blocked and the job is processed in the background. The client might also do some work to make it seem like it is completed. For example, a tweet could be posted to your timeline, even though it hasn't actually been delivered to all of your followers.

__Examples__

* Redis can act as simple message broker
* RabbitMQ

Message queues can become backfilled or full, and the asynchronism adds extra complexity and processing.

### RPCs

A client causes a procedure to execute on a different (usually) server. The call can be made as if it was a local procedure call, abstracting away a lot of the details.

* __Client program__: Calls the client stub procedure. The parameters are pushed onto the stack like a local procedure call.
* __Client stub procedure__: Marshals (packs) the procedure id and arguments into a request message.
* __Client communication module__: OS sends the message from client to server
* __Server communication module

![title](img/sd5.jpg)

## Design a REST API

__1. Consider the elements we have (nouns)__

E.g., users, messages, etc.

__2. Think of the relationships between them (verbs)__

This will give you the verbs you need.

Think in terms of CRUD operations:

* __Create__: POST
* __Read__: GET
* __Update__: PUT
* __Delete__: DELETE

__3. Create the requests__

These should be in the form `https://api-domain/version/collection/resource`

E.g.

* `/users/ GET` -- READS all users
* `/users/{ID} GET` -- READS a single user
* `/users/{ID} PUT`, Body: user data -- UPDATES a user
* `/users/{ID} POST`, Body: user data

For filtering, add parameters:

* `users/lastName=Smith GET`

For related resources:

* `users/{ID}/profile GET`

__4. Create the responses__

For responses, use JSON and HTTP status code:

```json

{
    "data": {
        "id": "123",
        "name": "John",
    }
}
```

| Code  | Use  | Example  |
|---    |---|---|
| 100   | Information  |   |
| 200   | Success  | OK  |
| 300   | Redirect  | Moved, temp redirect  |
| 400   | Client error  | Not found, unauthorized  |
| 500   | Server error  | Server error, unavailable  |

## Step-by-Step Guide

### LeetCode Template

__1. Feature Expectations__

* Use cases
* Who will use it
* How many users
* How do they use it

__2. Estimations__

* Throughput (QPS for reads and writes)
* Latency (speed for reads and writes)
* Read/write ratio
* Estimations for memory, cache

__3. Design Goals__

* Consistency vs. Availability (weak/strong/eventual > consistency, failover/replication > availability)

__4. High-level Design__

* APIs for read/write scenarios for crucial components
* Database schema
* Basic algorithms
* High level design for read/write scenarios

__5. Deep Dive__

* Scaling individual components
  * Availability / consistency / scalability for each component
  * Consistency and availability patterns
* Think about the following components:
  * DNS
  * CDN (push vs pull)
  * Load balancers (active/passive, active/active)
  * DB (RDBMS vs. NoSQL)
    * RDBMS
      * Master-slave, master-master, federation, sharding
    * NoSQL
      * Key-value, wide-column, document
        * Fast-lookups
        
   * Caches
     * Client-caching, CDN caching, server caching, DB caching
       * Cache-aside
   * Asynchronism
     * Message queues
     * Task queues
   * Communication
     * TCP
     * UDP
     * REST
     * RPC

__6. Justifications__
  * Throughput of each layer
  * Latency between each layer
  * Overall latency justification

### Summary from Notes

__1. Confirm: Use cases, constraints, assumptions__
  * Who is going to use the system
  * Platform: Mobile, desktop?
  * What are key features?
    * What are objects (DB) / verbs (services/API)
  * How many users are there?
  * How much data do we have?
  * What is the read/write ratio (which do we want to optimize for?)
  
__2. Create high-level design__
  * Sketch the main components and connections
  * Justify these ideas

__3. Core component design__
  * SQL vs NoSQL
  * DB Schema
  * API
  * Object design
    * Consider IDs too
    
__4. Scale & secure__

  * (Say you would load test, benchmark, profile)
  * Identify bottlenecks and single points of failure
  * Load balancers
  * Horizontal scaling
  * Caching
  * DB sharding


### 1. Ask clarifying questions

System design questions are usually not solvable within the time limit, so you must clarify which problem you are trying to solve. 4 main areas:

- __Users__: Who is going to use the system? (E.g. all users, 1 user, ML system, monthly marketing reports, etc.)
- __Scale__: What are the read / write requirements for data? (How many read queries need to be processed? How much data is processed per request? Can there be spikes?)
- __Performance__: How fast the system has to be? (What is the write-to-read data delay? Can we delay stat counting? How fast must data be retrieved from the system?)
- __Cost__: What budget constraints do we have? (Can we build our own, do we have maintenance budget? If not we could consider some managed cloud solution.)

### 2. Define Functional Requirements

Functional requirements refers to system behaviour, and specifically APIs / operations the system must support.

- Write down what the system must do, and turn those sentences into an API.
- These can be made more specific or generic as required.

For example:

_The system has to count video view events._

```python
countViewEvent(videoId)

-->

countEvent(videoId, eventType)

--> 

processEvent(videoId, eventType, aggregationMethod)  # Could be sum, avg, etc.

-->

processEvents(listOfEvents)
```

Likewise, we could do this for something like `getViews...getEventStats...getStats...`.

### 3. Define Non-Functional Requirements

How the system is supposed to be in terms of speed, security, and so on.
Normally the interviewer won't mention specific requirements. Instead they will say __high scale__ and __fast performance__. Because these two are hard to achieve at the same time, we will need to find tradeoffs.

- __Scalable__ (number of requests/second)
- __Performance__ (tens of milliseconds to return request)
- __Highly available__ (survives hardware outages, no single point of failure)

Also:

- __Consistency__
- __Cost__

### 4. High-level Design

This can be very simple:

![title](img/sd1.jpg)

Now the interviewer may jump into any of these components. As interviewee, it will be good if you can drive the conversation.

### 5. Detailed Design

#### 5.1. __What__ to store data

Store aggregate data, or individual (raw) data?

If it's okay to have a delay, we could store raw data and process it in the background (batch data processing), or if we need real-time, we need a pipeline to continuously aggregate data (stream data processing). We can also combine both approaches at the cost of money and complexity.

#### 5.2. __Where__ to store data

What kind of DB will you use? Look back at your non-functional requirements to understand the DB requirements. With each of our options / choices, we should look back see if it fulfills your non-functional requirements.

##### 5.2.1 SQL DBs

We _shard_ data over our databases, and the processing service needs to know which DB to query. 

We can introduce a proxy ('cluster proxy') to do this. We may also introduce a proxy ('shard proxy') to each DB so do things like report health, cache queries, etc.

We can back up each shard with replicas in different DBs. We can use these as read replicas.

![title](img/sd2.jpg)

##### 5.2.2 NoSQL DBs

(This example uses Cassandra, different NoSQL DBs work differently.)

We also split our data, but into __nodes__ (rather than shards). Each node is equal. And each node can exchange information about its state with other nodes. Clients can call any node in the server, and that node can forward to any other node without going through cluster proxy. Once it writes data to a node, that node can write replicas to other nodes. 

Now we might return before data has been fully replicated, which means some reads may get out of date data. To mitigate this, we can ensure data has been replicated to n nodes. (Here we see an application of CAP theorem.) 

![title](img/sd3.jpg)

###### Consider Consistency / Availability

Since replicating data is slow, do you want to wait for data to become available, or would you rather show some data ASAP, even if it is out of date?

#### 5.3. __How__ we store the data

For relational DBs:

* Define nouns, convert into tables and reference with keys

Eg., `videoStats`, `videoInfo`, `channelInfo` tables. To get a report, you join all tables by their keys.

Data is usually normalized to avoid duplication. We could also denormalize, which makes writes slower, but can speed up reads.

For non-relational DBs;

Everything you want is stored in a row. E.g.

`videoId, channelName, videoName, 1500-clicks, 1600-clicks`

Data is usually denormalized, to make queries fast.

Note there are 4 kinds of NoSQL DBs: column, document, key-value, graph.

#### Data Processing

##### 1. Get requirements and questions

- How to make data processing scalable, reliable, and fast?

* __Scalable__: DB partitioning
* __Reliable__: Replication and checkpointing
* __Fast__: In memory

- Do we want to pre-aggregate data, or process it in real-time?

#### 2. Fill in the details

Create the detailed design of how your service works. It could be things like `DB_Writer`. 

For example, create a queue of events and process them and save the results to the DB.

##### Blocking vs. Non-Blocking


###### Blocking Systems

In blocking systems, when a request comes in, we make a connection to the server via a socket, and that request is served by a single thread. It's easy to debug, but it means the thread gets blocked while processing and we can run out of connections.

###### Non-blocking Systems

Non-blocking systems don't block when a new request comes in. This can become hard to debug as a request is split over multiple threads.

##### Buffering & Batching

Instead of sending one event at a time, we should save our events in a buffer and send them in a batch together. This can save costs, be more efficient as we compress events, etc. However, it also adds complexity. For example, several events in the batch may fail, while some succeed.

##### Timeouts

__Connection timeouts__: We timed out waiting to get a connection. This is normally a very short timeout.

__Request timeouts__: We timed out serving the request. Maybe we just hit a bad machine. We can retry them. But be careful not to retry everything at once: use exponential backoff retries instead.

##### Load Balancing

__Software load balancing__: Can run on cloud or normal servers.

__Hardware load balancing__: Powerful, optimized to handle high throughput. Millions of requests per seconds.

Different load balancers can handle different types of traffic (TCP, HTTP, etc).

Different algorithms can be used to determine which server to send the request to:

* Hash-based (IP etc)
* Round robin
* Current load
* etc.

###### DNS

Translates domain names to IP addresses.
We register our partitioning service and associate it with the load balancer. The load balancer knows the IP addresses of the various servers running our service.

`Client (myservice.foo.com) -> DNS -> load balancer -> service servers`

###### Health Checking

The load balancer pings each server periodically to make sure it is still healthy.

###### Availability

We can have primary and secondary load balancers. If the secondary load balancer detects the primary load balancer is down, it takes over.

##### Message format

Do you use protobufs, XML, JSON, etc. Think about human-readable vs. smaller/faster.

##### Data Rollup / Retrieval

* Data for the last 7 days is stored at minute-by-minute granularity
* Data for the last 7-14 days is stored at hour-by-hour granularity
* etc.
* Older data can also be stored in cold storage, instead of hot storage
* Query results can be stored in a distributed cache

##### Technology Stack

Give some examples of technology you might use to build out this solution. For example, the DB, messaging systems, load balancer, cache you would use.

### 6. Evaluate Tradeoffs & find bottlenecks

Interview will likely start to question the design. This is where we can start to talk about the tradeoffs of your choices.

You may want to talk about load testing to verify the system is scalable and identify any bottle necks.

Stress testing could be used to identify where the system will break first -- for example, memory, DB, network, etc.

We could also add some health monitoring to make sure the system is always healthy.

We would also do testing to make sure it gives the correct results. We could build an audit system to do this. A weak audit system could run requests and make sure it gets the expected results. A strong audit system calculates stats using some totally different system (e.g. using MapReduce) and then check the results matches our system.